In [3]:
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from skrub import TableVectorizer
import utils
import best_params

In [2]:
X, y, X_test = utils.get_and_process_data()

/Users/gregzguegue/Python_Data_Challenge/utils.py:221: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/gregzguegue/Python_Data_Challenge/utils.py:221: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/gregzguegue/Python_Data_Challenge/utils.py:387: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are

In [4]:
# Initialisation des modèles
lgbm = LGBMRegressor(**best_params.parameters_LGBM)
rf = RandomForestRegressor(**best_params.parameters_RandomForest)
xgb = XGBRegressor(**best_params.parameters_XGBoost)

# Création du préprocesseur avec TableVectorizer
preprocessor = ColumnTransformer(
    transformers=[
        (
            "vectorizer",
            TableVectorizer(),
            X.columns,
        )  # Appliquer TableVectorizer à toutes les colonnes
    ]
)

# Création des pipelines pour les modèles
lgbm_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", lgbm)])
rf_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", rf)])
xgb_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", xgb)])

# Création du StackingRegressor
stacking_regressor = StackingRegressor(
    estimators=[("lgbm", lgbm_pipeline), ("rf", rf_pipeline), ("xgb", xgb_pipeline)],
    final_estimator=RandomForestRegressor(random_state=42),  # Meta-modèle
)

In [5]:
# Entraîner le modèle de stacking
stacking_regressor.fit(X, y)

KeyboardInterrupt: 

In [ ]:
submission = utils.test_fit_and_submission(X_test, stacking_regressor)